In [12]:
import cv2

base_path = "../data/resized_images/"
cam_views = ["Front", "Left", "Back", "Right"]
# Get paths to images from multi camera
image_paths = [base_path +
               cam_view + ".jpg" for cam_view in cam_views]
images = [cv2.imread(img, cv2.IMREAD_GRAYSCALE) for img in image_paths]
images

[array([[193, 192, 217, ..., 216, 217, 189],
        [203, 202, 217, ..., 228, 235, 229],
        [185, 205, 206, ..., 215, 185, 220],
        ...,
        [114, 115, 116, ..., 139, 130, 131],
        [106, 113, 123, ..., 193, 186, 185],
        [111, 111, 124, ..., 183, 167, 150]], shape=(571, 428), dtype=uint8),
 array([[175, 167, 169, ..., 240, 240, 237],
        [172, 166, 171, ..., 239, 238, 233],
        [168, 162, 166, ..., 235, 237, 232],
        ...,
        [ 59,  81,  60, ..., 157, 156, 143],
        [ 63,  85,  63, ..., 146, 150, 154],
        [ 61,  80,  58, ..., 140, 152, 164]], shape=(571, 428), dtype=uint8),
 array([[  5,   4,   5, ..., 182, 180, 171],
        [  4,   6,   7, ..., 197, 192, 189],
        [  6,   5,   6, ..., 140, 129, 125],
        ...,
        [ 31,  32,  29, ..., 185, 191, 197],
        [ 41,  50,  56, ..., 188, 188, 189],
        [ 47,  63,  75, ..., 128, 125, 122]], shape=(571, 428), dtype=uint8),
 array([[ 76,  90,  83, ...,   7,   6,   7],
       

In [6]:
# Initialize feature detector (ORB or SIFT)
use_sift = True
if use_sift:
    feature_detector = cv2.SIFT_create()
    bf = cv2.BFMatcher(cv2.NORM_L2, crossCheck=False)  # SIFT uses L2 norm
else:
    feature_detector = cv2.ORB_create()
    # ORB uses Hamming norm
    bf = cv2.BFMatcher(cv2.NORM_HAMMING, crossCheck=True)

# Detect keypoints and descriptors
keypoints_descriptors = [
    feature_detector.detectAndCompute(img, None) for img in images]
keypoints = [kp_desc[0] for kp_desc in keypoints_descriptors]
descriptors = [kp_desc[1] for kp_desc in keypoints_descriptors]
len(descriptors)

4

In [7]:
# Function to match features between two images
def match_features(desc1, desc2):
    matches = bf.knnMatch(desc1, desc2, k=2) if use_sift else bf.match(desc1, desc2)
    
    # Apply Lowe's ratio test for SIFT
    if use_sift:
        good_matches = [m for m, n in matches if m.distance < 0.75 * n.distance]
    else:
        good_matches = matches  # ORB uses crossCheck, so matches are already good
    
    return len(good_matches), good_matches

# Compare matches between consecutive images
for i in range(len(images) - 1):
    if descriptors[i] is not None and descriptors[i+1] is not None:
        num_matches, good_matches = match_features(descriptors[i], descriptors[i+1])
        print(f"Matches between {image_paths[i]} and {image_paths[i+1]}: {num_matches}")
    else:
        print(f"Skipping {image_paths[i]} and {image_paths[i+1]} due to missing descriptors.")

Matches between ../data/resized_images/Front.jpg and ../data/resized_images/Left.jpg: 18
Matches between ../data/resized_images/Left.jpg and ../data/resized_images/Back.jpg: 19
Matches between ../data/resized_images/Back.jpg and ../data/resized_images/Right.jpg: 49


In [8]:
# Check keypoints before visualization
img_idx = 0  # Change to visualize a different image pair
if keypoints[img_idx] and keypoints[img_idx + 1]:
    num_matches, good_matches = match_features(
        descriptors[img_idx], descriptors[img_idx + 1])

    if good_matches:  # Ensure there are matches before drawing
        img_matches = cv2.drawMatches(
            images[img_idx], keypoints[img_idx],
            images[img_idx + 1], keypoints[img_idx + 1],
            good_matches, None, flags=cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS
        )
        cv2.imshow("Matches", img_matches)
        cv2.waitKey(0)
        cv2.destroyAllWindows()
    else:
        print("No good matches found to visualize.")
else:
    print("Not enough keypoints for visualization.")